In [1]:
from snowflake.snowpark import Session
import pandas as pd
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, recall_score, precision_score

In [2]:
connection_parameters = {
    "account": "wkwmqzq-hb12552",
    "user": "Dikshit",
    "password": "Sagar@123",
    "role": "ACCOUNTADMIN",
    "warehouse": "COMPUTE_WH",
    "database": "DIKSHIT",
    "schema": "SAGAR"
  }


In [3]:
def new_sss():

    new_session = Session.builder.configs(connection_parameters).create()
    return new_session
session = new_sss()
session

In [105]:
session.add_packages(['xgboost'] )

In [4]:
df = session.sql(query= "select * from snowflake_sample_data.TPCDS_SF100TCL.CUSTOMER where C_BIRTH_DAY = 8")

In [5]:
df.limit(2).to_pandas()

,C_CUSTOMER_SK,C_CUSTOMER_ID,C_CURRENT_CDEMO_SK,C_CURRENT_HDEMO_SK,C_CURRENT_ADDR_SK,C_FIRST_SHIPTO_DATE_SK,C_FIRST_SALES_DATE_SK,C_SALUTATION,C_FIRST_NAME,C_LAST_NAME,C_PREFERRED_CUST_FLAG,C_BIRTH_DAY,C_BIRTH_MONTH,C_BIRTH_YEAR,C_BIRTH_COUNTRY,C_LOGIN,C_EMAIL_ADDRESS,C_LAST_REVIEW_DATE
0,3879930,AAAAAAAAKPDDLDAA,1070725,4780,2929682,2450284,2450254,Mr.,Scott,Bernard,Y,8,11,1928,MADAGASCAR,None,Scott.Bernard@JX95DlhkDeEC.com,2452619
1,3879992,AAAAAAAAIDEDLDAA,1182329,5973,27569170,2451419,2451389,Sir,Darrell,Jones,Y,8,5,1979,GABON,None,Darrell.Jones@hhoIHD4fihg.com,2452576


In [42]:
new_session.sql(query= "select count(*) as count from snowflake_sample_data.TPCDS_SF100TCL.CUSTOMER where C_CUSTOMER_ID = 'AAAAAAAAKPDDLDAA'").show()

-----------
|"COUNT"  |
-----------
|1        |
-----------



In [18]:
new_session.sql(query= "select distinct C_BIRTH_COUNTRY, count(*) from snowflake_sample_data.TPCDS_SF100TCL.CUSTOMER group by C_BIRTH_COUNTRY").show()

----------------------------------
|"C_BIRTH_COUNTRY"  |"COUNT(*)"  |
----------------------------------
|SWITZERLAND        |456874      |
|GEORGIA            |458212      |
|MALI               |458500      |
|LITHUANIA          |457885      |
|CANADA             |458240      |
|MALDIVES           |456754      |
|LIBERIA            |458528      |
|AUSTRALIA          |457319      |
|MALAWI             |456894      |
|SUDAN              |457109      |
----------------------------------



Customers by birth country

In [36]:
view_name_1 = "Cust_by_country_v"
view_name_2 = "Cust_by_Year_v"
view_name_3 = "Join_view"
database = "DIKSHIT"
schema = "SAGAR"

In [21]:
df2 = new_session.sql(query= "select * from snowflake_sample_data.TPCDS_SF100TCL.CUSTOMER where C_BIRTH_COUNTRY = 'FINLAND'")
df2.create_or_replace_view(f"{database}.{schema}.{view_name_1}")

[Row(status='View CUST_BY_COUNTRY_V successfully created.')]

Customers by Birth Year greater than 1950

In [24]:
df3 = new_session.sql(query= "select * from snowflake_sample_data.TPCDS_SF100TCL.CUSTOMER where C_BIRTH_YEAR > '1950'")
df3.create_or_replace_view(f"{database}.{schema}.{view_name_2}")

[Row(status='View CUST_BY_YEAR_V successfully created.')]

Joining and saving in a snowflak view 


In [38]:
df4 = new_session.sql(query= "select a.*,b.* from snowflake_sample_data.TPCDS_SF100TCL.CUSTOMER a left join snowflake_sample_data.TPCDS_SF100TCL.CUSTOMER_ADDRESS b \
                                 on a.C_CUSTOMER_ID = b.CA_ADDRESS_ID ")
df4.create_or_replace_view(f"{database}.{schema}.{view_name_3}")


[Row(status='View JOIN_VIEW successfully created.')]

Calling Snowpark Function from snowpark

In [61]:
new_session.sql(query = "select * from table(DIKSHIT.SAGAR.get_data('AAAAAAAAKPDDLDAA'))").show()

----------------------------------
|"C_FIRST_NAME"  |"C_LAST_NAME"  |
----------------------------------
|Scott           |Bernard        |
----------------------------------



Writing DataFrame to Snowflake table

In [69]:
df.write.mode('overwrite').saveAsTable('DIKSHIT.SAGAR.CUSTOMERs');

In [ ]:
from snowflake.snowpark.types import IntegerType
from snowflake.snowpark.functions import udf
add_one = new_session.udf.register(lambda x: x+1, return_type=IntegerType(),is_permanent=True , stage_location= "@UDF1/",  input_types=[IntegerType()], name="my_udf", replace=True)

In [78]:
new_session.sql("select UDF1()").show()

------------
|"UDF1()"  |
------------
|3.141     |
------------

